In [1]:
#Importing necessary libs
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
#Reading file
oscar_df = pd.read_csv('oscar.csv', sep='\t')

In [3]:
# Lets see whats the data we're working with
oscar_df.head()

,normal_query,datetime
0,neityfz говядина как консервы,2019-01-23 09:39:25
1,водокаал алексин,2019-01-23 09:39:25
2,настя шевцова и гуф,2019-01-23 09:39:25
3,ул барсова 7 сочи,2019-01-23 09:39:25
4,доп соглашение к срочному трудовому договору о...,2019-01-23 09:39:25


In [4]:
# At 1st, let's transform the 'datetime' column into datetime format
oscar_df['datetime'] = pd.to_datetime(oscar_df['datetime'])

# Then let's divide whole DF into 3 parts, according to Oscar ceremony date (2019-02-25): before/at/after parts; 
# and cleaning the data from missing walues at the same time 
oscar_df_before = oscar_df[oscar_df['datetime'] < pd.to_datetime('2019-02-25')].dropna(axis = 0)
oscar_df_at = oscar_df[oscar_df['datetime'] == pd.to_datetime('2019-02-25')].dropna(axis = 0)
oscar_df_after = oscar_df[oscar_df['datetime'] > pd.to_datetime('2019-02-26')].dropna(axis = 0)

In [10]:
# Lets see the size of each part
print("Part 'Before Oscar'", oscar_df_before.shape[0], "rows"), 
print("Part 'At Oscar'", oscar_df_at.shape[0], "rows"),
print("Part 'After Oscar'", oscar_df_after.shape[0], "rows")

Part 'Before Oscar' 34616468 rows
Part 'At Oscar' 6 rows
Part 'After Oscar' 3193689 rows


In [11]:
# In order to feed the CountVectorizer with data correctly, lets take only the 'normal query' columns 
# and transform it into lists of queries   
oscar_lst_b = oscar_df_before['normal_query'].tolist()
oscar_lst_at = oscar_df_at['normal_query'].tolist()
oscar_lst_af = oscar_df_after['normal_query'].tolist()

**Lets prepare data for LDA. Lets make sparse matrices from each part**

In [14]:
%%time
n_features = 1000
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features = n_features)

tf_b = tf_vectorizer.fit_transform(oscar_lst_b)
tf_feature_names_b = tf_vectorizer.get_feature_names()

Wall time: 3min 1s


In [16]:
%%time
n_features = 1000
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=1, max_features = n_features)

tf_at = tf_vectorizer.fit_transform(oscar_lst_at)
tf_feature_names_at = tf_vectorizer.get_feature_names()

Wall time: 0 ns


In [18]:
%%time
n_features = 1000
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features = n_features)

tf_af = tf_vectorizer.fit_transform(oscar_lst_af)
tf_feature_names_af = tf_vectorizer.get_feature_names()

Wall time: 18.8 s


**Lets fit the LDA to each part separatly and see what time it will take**

In [15]:
%%time
n_topics = 5
lda_b = LatentDirichletAllocation(n_components=n_topics,\
                                  max_iter=5, learning_method='online',\
                                  learning_offset=50.0,random_state=84).fit(tf_b)

Wall time: 2h 15min 24s


In [20]:
%%time
n_topics = 5
lda_at = LatentDirichletAllocation(n_components=n_topics,\
                                  max_iter=5, learning_method='online',\
                                  learning_offset=50.0,random_state=84).fit(tf_at)

Wall time: 13 ms


In [24]:
%%time
n_topics = 5
lda_af = LatentDirichletAllocation(n_components=n_topics,\
                                  max_iter=5, learning_method='online',\
                                  learning_offset=50.0,random_state=84).fit(tf_af)

Wall time: 11min 9s


In [28]:
# Making function for the output
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

**Lets group all in 5 topics for each of 3 parts**

Topics of the queries was made <font color = 'green'> **BEFORE** </font> the day of Oscar ceremony

In [25]:
n_top_words = 10
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda_b, tf_feature_names_b, n_top_words)

Topic #0: не видео вк одноклассники страница про моя где москва дом
Topic #1: порно это сезон 10 секс магазин серия февраля интернет песня
Topic #2: на для онлайн купить 2019 2018 отзывы по за цена
Topic #3: как на сайт официальный что из от ру ru сколько
Topic #4: скачать по смотреть фильм на бесплатно класс фото погода сериал



Topics of the queries was made <font color = 'green'> **AT** </font> the day of Oscar ceremony

In [23]:
n_top_words = 10
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda_at, tf_feature_names_at, n_top_words)

Topic #0: самара на погода катя преследует индивидуум gta уилл womeximizer адушкина
Topic #1: катя адушкина womeximizer gta феррелл уилл самара индивидуум каждый английском
Topic #2: уилл womeximizer феррелл gta самара на английском индивидуум преследует погода
Topic #3: gta womeximizer индивидуум преследует каждый катя феррелл погода английском адушкина
Topic #4: английском преследует каждый индивидуум погода на катя феррелл самара уилл



Topics of the queries was made <font color = 'green'> **AFTER** </font> the day of Oscar ceremony

In [29]:
n_top_words = 10
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda_af, tf_feature_names_af, n_top_words)

Topic #0: порно сайт официальный отзывы ру видео вк авито секс страница
Topic #1: для скачать онлайн смотреть бесплатно все без области спб качестве
Topic #2: по как класс купить что от это гдз россии про
Topic #3: на из за при цена погода ли можно работа сегодня
Topic #4: 2019 фильм 2018 фото не ru сезон сколько сериал 10

